<a href="https://colab.research.google.com/github/spps-supalerk/ThaiGovernmentData/blob/main/CleanThaiGovernmentData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive2 = GoogleDrive(gauth)

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Thailand : Thai Population by Tambon (2562)
Link : https://stat.bora.dopa.go.th/new_stat/webPage/statByYear.php

In [3]:
df_pop = pd.read_excel('/content/drive/MyDrive/ThaiME/ExternalData/population_62.xlsx')

df_pop.head()

,YYMM,ProvinceCode,Province,AmphurCode,Amphur,TambonCode,Tambon,Male,Female,Total,House
0,6312,0,ทั่วประเทศ,0,,0,,32375532,33811195,66186727,27224743
1,6312,10,กรุงเทพมหานคร,0,,0,,2625938,2962284,5588222,3103483
2,6312,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,0,,21675,23248,44923,19137
3,6312,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,10010100,พระบรมมหาราชวัง,1977,1369,3346,1207
4,6312,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,10010200,วังบูรพาภิรมย์,4880,4770,9650,5363


In [4]:
# filter sub-total while contains 0 in TabonCode column
df_pop = df_pop[df_pop['TambonCode']!=0]

# remove 'อำเภอ' in column Amphur
df_pop['Amphur'] = df_pop['Amphur'].str.replace('อำเภอ','')

# select needed column
lst_col = ['Province', 'Amphur', 'Tambon', 'Total', 'House']
df_pop_cleaned = df_pop[lst_col]

# change column name
df_pop_cleaned.columns = ['province', 'district', 'subdistrict', 'population_62', 'household_62']
df_pop_cleaned.head()

,province,district,subdistrict,population_62,household_62
3,กรุงเทพมหานคร,ท้องถิ่นเขตพระนคร,พระบรมมหาราชวัง,3346,1207
4,กรุงเทพมหานคร,ท้องถิ่นเขตพระนคร,วังบูรพาภิรมย์,9650,5363
5,กรุงเทพมหานคร,ท้องถิ่นเขตพระนคร,วัดราชบพิธ,3071,979
6,กรุงเทพมหานคร,ท้องถิ่นเขตพระนคร,สำราญราษฎร์,2940,1127
7,กรุงเทพมหานคร,ท้องถิ่นเขตพระนคร,ศาลเจ้าพ่อเสือ,2959,999


In [5]:
df_pop_cleaned.to_csv('/content/drive/MyDrive/ThaiME/ExternalData/clean_population_62.csv',index=False)

# Thailand : Average Income by Province (2562)
Link : http://statbbi.nso.go.th/staticreport/page/sector/th/02.aspx

In [6]:
df_avg = pd.read_excel('/content/drive/MyDrive/ThaiME/ExternalData/avg_income_62.xlsx',
                       skiprows=2)
df_avg.head()

,ภาค,จังหวัด,2545,2547,2549,2550,2552,2554,2556,2558,2560,2562
0,ทั่วราชอาณาจักร,ทั่วราชอาณาจักร,13736.00,14963.40,17787.0,18660.0,20903.000000,23236.0,25194.0,26915.0,26946.43,26018.42
1,กรุงเทพมหานคร และ 3 จังหวัด,กรุงเทพมหานคร และ 3 จังหวัด,28238.66,28135.36,33088.0,35007.0,37732.000000,41631.0,43058.0,41002.0,41897.06,37750.96
2,NaN,กรุงเทพมหานคร,29589.08,29842.68,36658.0,39020.0,42379.830480,48951.0,49190.8,45571.7,45707.31,39459.36
3,NaN,สมุทรปราการ,19679.94,19946.54,20382.0,21302.0,23359.377965,23797.9,29575.3,25457.2,28711.77,24729.42
4,NaN,นนทบุรี,29119.13,26657.97,31152.0,32743.0,34626.276126,35119.7,30663.6,36884.0,40860.88,37502.22


In [7]:
## ffill
df_avg['ภาค'] = df_avg['ภาค'].fillna(method ='ffill')

## drop source (last row)
df_avg = df_avg.dropna(subset=['จังหวัด'])

# drop sub-total
# จังหวัด not in below_lst
exclude_lst = ['ทั่วราชอาณาจักร',
               'กรุงเทพมหานคร และ 3 จังหวัด',
               'ภาคกลาง (ไม่รวม กทม. และ 3 จังหวัดรอบ กทม.)',
               'ภาคเหนือ',
               'ภาคตะวันออกเฉียงเหนือ',
               'ภาคใต้']
df_avg = df_avg[~df_avg['จังหวัด'].isin(exclude_lst)]

# replace ['กรุงเทพมหานคร และ 3 จังหวัด', 'ภาคกลาง (ไม่รวม กทม. และ 3 จังหวัดรอบ กทม.)'] with ภาคกลาง
df_avg['ภาค'] = df_avg['ภาค'].replace(['กรุงเทพมหานคร และ 3 จังหวัด', 'ภาคกลาง (ไม่รวม กทม. และ 3 จังหวัดรอบ กทม.)'], 'ภาคกลาง')

# keep only year 2562
df_avg_cleaned = df_avg[['ภาค','จังหวัด',2562]]

# rename columns
df_avg_cleaned.rename(columns = {'ภาค':'region',
                                 'จังหวัด':'province',
                                 2562:'avg_income_62'},
                      inplace=True)

df_avg_cleaned.head()

,region,province,avg_income_62
2,ภาคกลาง,กรุงเทพมหานคร,39459.36
3,ภาคกลาง,สมุทรปราการ,24729.42
4,ภาคกลาง,นนทบุรี,37502.22
5,ภาคกลาง,ปทุมธานี,46977.74
7,ภาคกลาง,พระนครศรีอยุธยา,30589.95


In [8]:
df_avg_cleaned.to_csv('/content/drive/MyDrive/ThaiME/ExternalData/clean_avg_income_62.csv', index=False)

# Thailand : Labor Force by Province (2562)
Link : http://statbbi.nso.go.th/staticreport/page/sector/th/02.aspx

In [9]:
df_labor = pd.read_excel('/content/drive/MyDrive/ThaiME/ExternalData/unemployment_62.xlsx',
                         skiprows=4)
df_labor.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,ไตรมาสที่ 1,Unnamed: 6,Unnamed: 7,ไตรมาสที่ 2,Unnamed: 9,Unnamed: 10,ไตรมาสที่ 3,Unnamed: 12,Unnamed: 13,ไตรมาสที่ 4,Unnamed: 15,Unnamed: 16,เฉลี่ยทั้งปี,Unnamed: 18,Unnamed: 19
0,ภาค,จังหวัด,สถานภาพแรงงาน,กำลังคน,กำลังแรงงานปัจจุบัน,รวม,ชาย,หญิง,รวม,ชาย,หญิง,รวม,ชาย,หญิง,รวม,ชาย,หญิง,รวม,ชาย,หญิง
1,ทั่วราชอาณาจักร,ทั่วราชอาณาจักร,ประชากรอายุ 15 ปีขึ้นไป,NaN,NaN,5.64652e+07,2.72529e+07,29212239,56541221,2.72859e+07,2.92553e+07,5.66157e+07,2.7319e+07,29296725,5.66789e+07,27346596,2.93323e+07,5.65753e+07,2.73011e+07,2.92742e+07
2,NaN,NaN,กำลังแรงงานรวม,NaN,NaN,3.83652e+07,2.08068e+07,1.75583e+07,3.84201e+07,2.0916e+07,1.75041e+07,3.79587e+07,2.07509e+07,1.72078e+07,3.79682e+07,2.07063e+07,1.72619e+07,3.8178e+07,2.0795e+07,1.7383e+07
3,NaN,NaN,NaN,กำลังแรงงานปัจจุบัน,NaN,3.80539e+07,2.06293e+07,1.74246e+07,3.81587e+07,2.07652e+07,1.73935e+07,3.78804e+07,2.06935e+07,1.71868e+07,3.78544e+07,2.0627e+07,1.72274e+07,3.79868e+07,2.06787e+07,1.73081e+07
4,NaN,NaN,NaN,NaN,ผู้มีงานทำ,3.77027e+07,2.04288e+07,1.72739e+07,3.77818e+07,2.05609e+07,1.72209e+07,3.74863e+07,2.04675e+07,1.70188e+07,3.74829e+07,2.04082e+07,1.70747e+07,3.76134e+07,2.04664e+07,1.71471e+07


In [10]:
df_labor_clean_col = df_labor.loc[0,:].reset_index()
labor_new_col = df_labor_clean_col.apply(lambda x: x['index']+'_'+x[0] if 'Unnamed' not in x['index'] else x[0], axis=1)
labor_new_col

0                     ภาค
1                 จังหวัด
2           สถานภาพแรงงาน
3                 กำลังคน
4     กำลังแรงงานปัจจุบัน
5         ไตรมาสที่ 1_รวม
6                     ชาย
7                    หญิง
8         ไตรมาสที่ 2_รวม
9                     ชาย
10                   หญิง
11        ไตรมาสที่ 3_รวม
12                    ชาย
13                   หญิง
14        ไตรมาสที่ 4_รวม
15                    ชาย
16                   หญิง
17       เฉลี่ยทั้งปี_รวม
18                    ชาย
19                   หญิง
dtype: object

In [11]:
# assign new column
df_labor.columns = labor_new_col 

# ignore first row
df_lober = df_labor.loc[1:,:]
df_lober.head()

,ภาค,จังหวัด,สถานภาพแรงงาน,กำลังคน,กำลังแรงงานปัจจุบัน,ไตรมาสที่ 1_รวม,ชาย,หญิง,ไตรมาสที่ 2_รวม,ชาย,หญิง,ไตรมาสที่ 3_รวม,ชาย,หญิง,ไตรมาสที่ 4_รวม,ชาย,หญิง,เฉลี่ยทั้งปี_รวม,ชาย,หญิง
1,ทั่วราชอาณาจักร,ทั่วราชอาณาจักร,ประชากรอายุ 15 ปีขึ้นไป,NaN,NaN,5.64652e+07,2.72529e+07,29212239,56541221,2.72859e+07,2.92553e+07,5.66157e+07,2.7319e+07,29296725,5.66789e+07,27346596,2.93323e+07,5.65753e+07,2.73011e+07,2.92742e+07
2,NaN,NaN,กำลังแรงงานรวม,NaN,NaN,3.83652e+07,2.08068e+07,1.75583e+07,3.84201e+07,2.0916e+07,1.75041e+07,3.79587e+07,2.07509e+07,1.72078e+07,3.79682e+07,2.07063e+07,1.72619e+07,3.8178e+07,2.0795e+07,1.7383e+07
3,NaN,NaN,NaN,กำลังแรงงานปัจจุบัน,NaN,3.80539e+07,2.06293e+07,1.74246e+07,3.81587e+07,2.07652e+07,1.73935e+07,3.78804e+07,2.06935e+07,1.71868e+07,3.78544e+07,2.0627e+07,1.72274e+07,3.79868e+07,2.06787e+07,1.73081e+07
4,NaN,NaN,NaN,NaN,ผู้มีงานทำ,3.77027e+07,2.04288e+07,1.72739e+07,3.77818e+07,2.05609e+07,1.72209e+07,3.74863e+07,2.04675e+07,1.70188e+07,3.74829e+07,2.04082e+07,1.70747e+07,3.76134e+07,2.04664e+07,1.71471e+07
5,NaN,NaN,NaN,NaN,ผู้ว่างงาน,351157,200437,150720,376926,204338,172589,394026,226018,168008,371477,218742,152735,373396,212384,161013


In [12]:
# forward fill missing data
df_labor['จังหวัด'] = df_labor['จังหวัด'].fillna(method='ffill')

# select only ['ผู้มีงานทำ', 'ผู้ว่างงาน']
df_labor = df_labor[df_labor['กำลังแรงงานปัจจุบัน'].isin(['ผู้มีงานทำ', 'ผู้ว่างงาน'])]

# select column in list
col = ['จังหวัด','กำลังแรงงานปัจจุบัน', 'ไตรมาสที่ 4_รวม']
df_labor_cleaned = df_labor[col]

In [13]:
# change data type
df_labor_cleaned['ไตรมาสที่ 4_รวม'] = df_labor_cleaned['ไตรมาสที่ 4_รวม'].astype(float)

# ignore sub total
lst_sub_total = ['ทั่วราชอาณาจักร', 'ภาคกลาง', 'ภาคตะวันออกเฉียงเหนือ', 'ภาคเหนือ', 'ภาคใต้']
df_labor_cleaned = df_labor_cleaned[~df_labor_cleaned['จังหวัด'].isin(lst_sub_total)]

# create singleview

df_labor_cleaned_pv = df_labor_cleaned.pivot_table(index='จังหวัด', columns = 'กำลังแรงงานปัจจุบัน', values = 'ไตรมาสที่ 4_รวม', aggfunc='sum').reset_index()
df_labor_cleaned_pv.columns = ['province', 'employ_62', 'unemploy_62']
df_labor_cleaned_pv.head()

,province,employ_62,unemploy_62
0,กระบี่,212865.74,3648.87
1,กรุงเทพมหานคร,5264292.95,32313.31
2,กาญจนบุรี,463672.98,1885.19
3,กาฬสินธุ์,409106.41,321.65
4,กำแพงเพชร,405775.09,5842.89


In [14]:
df_labor_cleaned_pv.to_csv('/content/drive/MyDrive/ThaiME/ExternalData/clean_labor_force_62.csv', index=False)

# Thailand : Poor People by Province (2562)
Link : http://statbbi.nso.go.th/staticreport/page/sector/th/08.aspx

In [15]:
# unit = 1k
df_poor = pd.read_excel('/content/drive/MyDrive/ThaiME/ExternalData/poor_people_62.xlsx',
                        skiprows=3)
df_poor.head()

,ภาค,จังหวัด,2553,2554,2555,2556,2557,2558,2559,2560,2561,2562,Unnamed: 12
0,ทั่วราชอาณาจักร,ทั่วราชอาณาจักร,10800.728130,8751.881803,8402.075638,7305.1,7057.4,4847.2,5810.1,5324.8,6682.5,4326.152000,NaN
1,กรุงเทพมหานคร,กรุงเทพมหานคร,186.659136,647.923425,161.460953,90.4,140.6,173.7,118.0,98.3,123.6,52.907190,NaN
2,ภาคกลาง,ภาคกลาง,1957.854996,1905.133996,1291.522012,1014.9,941.5,827.3,1010.4,884.0,1374.9,883.013500,NaN
3,NaN,สมุทรปราการ,27.711701,34.647418,16.811365,20.1,20.2,7.5,6.0,0.8,7.0,12.429620,NaN
4,NaN,นนทบุรี,17.792057,16.387578,9.113487,11.9,2.8,9.2,10.6,11.5,11.8,4.201143,NaN


In [16]:
# select columns
col = ['จังหวัด',2562]
df_poor = df_poor[col]

# drop null values in จังหวัด 
df_poor.dropna(subset=['จังหวัด'], inplace=True)

# ignore sub total
lst_sub_total = ['ทั่วราชอาณาจักร', 'ภาคกลาง', 'ภาคตะวันออกเฉียงเหนือ', 'ภาคเหนือ', 'ภาคใต้']
df_poor_cleaned = df_poor[~df_poor['จังหวัด'].isin(lst_sub_total)]

# rescale and store in poor_people_62 
df_poor_cleaned['poor_people_62'] = df_poor_cleaned[2562] * 1000

# rename column
df_poor_cleaned.rename(columns={'จังหวัด':'province'}, inplace=True)

# drop unneeded column
df_poor_cleaned.drop(columns=2562, inplace=True)

df_poor_cleaned.head()

,province,poor_people_62
1,กรุงเทพมหานคร,52907.190
3,สมุทรปราการ,12429.620
4,นนทบุรี,4201.143
5,ปทุมธานี,4253.961
6,พระนครศรีอยุธยา,19946.500


In [17]:
df_poor_cleaned.to_csv('/content/drive/MyDrive/ThaiME/ExternalData/clean_poor_people.csv', index=False)

# Merge 3 dfs Together
## df_avg_cleaned, df_labor_cleaned_pv, df_poor_cleaned (province lv)

In [25]:
# merge df_avg_cleaned, df_labor_cleaned_pv, df_poor_cleaned
df_province_char = df_avg_cleaned.merge(df_labor_cleaned_pv, how='outer', on='province')
df_province_char = df_province_char.merge(df_poor_cleaned, how='outer', on='province')
df_province_char.isnull().sum()

region            0
province          0
avg_income_62     0
employ_62         0
unemploy_62       0
poor_people_62    0
dtype: int64

In [26]:
df_province_char.head()

,region,province,avg_income_62,employ_62,unemploy_62,poor_people_62
0,ภาคกลาง,กรุงเทพมหานคร,39459.36,5264292.95,32313.31,52907.190
1,ภาคกลาง,สมุทรปราการ,24729.42,1349805.86,13941.48,12429.620
2,ภาคกลาง,นนทบุรี,37502.22,956435.38,7715.10,4201.143
3,ภาคกลาง,ปทุมธานี,46977.74,949136.36,15237.97,4253.961
4,ภาคกลาง,พระนครศรีอยุธยา,30589.95,486961.34,10211.69,19946.500


In [27]:
df_province_char.to_csv('/content/drive/MyDrive/ThaiME/ExternalData/full_province_char.csv', index=False)